# Import Necessary Modules

In [ ]:
import lasio as ls
import os

# Create a List of All .las Files in Folder

In [ ]:
las_file_list = []
path = 'Input Folder' # Paste Input Folder path containing .las files

files = os.listdir(path)

for file in files:
    if file.lower().endswith('.las'):
        las_file_list.append(path + '\\' + file)

# Parse Through All .las Files to Modify API Numbers
* Code accounts for no API in header by creating an API from the reported UWI, if available
* Additional If/Elif statements can be incorporated based on API formatting
* Current version overwrites existing file
* Exceptions will allow you to run through certain errors.  File name associated with those files with errors (and not fixed) will output below

In [ ]:
for las in las_file_list:
    try:
        
        fname = os.path.basename(las)
        las = ls.read(las)
        
    except Exception:
        print('Some Other Error for:', fname)
        continue  
    
    try:
        las.well.API
    except AttributeError:
        try:
            las.well.API = ls.HeaderItem(mnemonic='API', value=las.well.UWI.value, descr='API')
        except AttributeError:
            print('No API or UWI Found for:', fname)
          
    try:

        if '-' in las.well.API.value and len(las.well.API.value) == 12:
            las.well.API = str(int(las.well.API.value.replace('-',''))*10000) 

        elif '-' in las.well.API.value and len(las.well.API.value) == 17:
            las.well.API = str(int(las.well.API.value.replace('-',''))) 

        elif len(las.well.API.value) == 12:
            las.well.API = str(int(las.well.API.value) * 100) 

        elif len(las.well.API.value) == 10:
            las.well.API = str(int(las.well.API.value) * 10000)
    except Exception:
        print('Some Other Error for:', fname)
        continue
               
    with open('Output Folder' + fname, mode='w') as f: # Insert Output Folder path at Output Folder location
        las.write(f, version=2.0)